# Beginning TeleOperations

The goal with this activity is to remotely pilot your smartphone around.  You will need to build a smartphone holder and Kate and Maddie will take your robots and toss them somewhere in Anderson.  The first group to remotely drive to my office will win.

<h4>Constraints:</h4>

- the bases will start in an undisclosed location
- you will have to zoom into your phone with your teammate's phone to see where you are going
- you will need to be able to remotely control your robot's motion

<h4>Grading rubric:</h4>

6. you came to class
7. you made an attempt: something happened - the robot moved
8. satisfactory: you drove your robot around in a remote location
9. good: your robot easily moved around in the remote location and you designed and built an elegant smartphone holder.
10. excellent: you added functionality to the keyboard controller, and you had a nice phone holder.
11. above and beyond: totally up to you

<h4>Your code </h4>

You can edit and improve on existing code for this - or use as is.  The first step is to make sure you are talking to your Create.  Try going [here](../Help/201b_FindCreate.ipynb) if you are having issues.

In [ ]:
import os
os.environ['ROS_DOMAIN_ID']="1"
os.environ['RMW_IMPLEMENTATION']="rmw_cyclonedds_cpp"

#try a few times to get the topic list
for i in range(10):
    list = subprocess.run(['ros2','topic','list'],capture_output=True)
    if len(list.stdout.decode().split('\n')) > 4:
        print(list.stdout.decode())
        break
    time.sleep(1)
if i >=9: 
    print('connection failed')


Again you will have to run the code in the terminal: open up a terminal from the launcher, reset your Domain ID, and try 

*python3.8 TeleOp.py*


In [ ]:
%%writefile TeleOp.py

# Stolen (and edited) from Create examples - https://github.com/iRobotEducation/create3_examples

import sys

import termios
import tty
from Subs.CreateLib import Create

msg = '''
---------------------------
Moving around:
   u    i    o
   j    k    l
   m    ,    .

anything else : stop

CTRL-C to quit
'''

moveOptions = {  #  x, y, z velocity, angle velocity (radians/sec)
    'u': (1, 0, 0, 1),
    'i': (1, 0, 0, 0),
    'o': (1, 0, 0, -1),
    
    'j': (0, 0, 0, 1),
    'l': (0, 0, 0, -1),

    'm': (-1, 0, 0, -1),  
    ',': (-1, 0, 0, 0),
    '.': (-1, 0, 0, 1),
}

def getKey(settings):
    tty.setraw(sys.stdin.fileno())
    # sys.stdin.read() returns a string on Linux
    key = sys.stdin.read(1)
    termios.tcsetattr(sys.stdin, termios.TCSADRAIN, settings)
    return key

def saveTerminalSettings():
    return termios.tcgetattr(sys.stdin)

def restoreTerminalSettings(old_settings):
    termios.tcsetattr(sys.stdin, termios.TCSADRAIN, old_settings)

def main():
    settings = saveTerminalSettings()
    motion = Create('/rogers')

    speed = 0.5
    turn = 1.0
    (x, y, z, th) = (0.0,0.0,0.0,0.0)

    try:
        print(msg)
        print('currently:  speed %s  turn %s ' % (speed, turn))
        
        while True:
            key = getKey(settings)
            if key in moveOptions.keys():
                x = moveOptions[key][0]
                y = moveOptions[key][1]
                z = moveOptions[key][2]
                th = moveOptions[key][3]
                
                motion.twist(x, y, z, th, speed, turn)
                
            else:
                (x, y, z, th) = (0.0,0.0,0.0,0.0)
                if (key == '\x03'):
                    break
                    
    except Exception as e:
        print(e)

    finally:
        motion.twist(0.0, 0.0, 0.0, 0.0, 0.5, 1.0)
        restoreTerminalSettings(settings)

if __name__ == '__main__':
    main()


<h4>Your reflections </h4>

Type your thoughts here - what worked well - what did not?  What was tough and what was easy?  Share any good links you found that helped.

<h4>More thoughts</h4>

You can try starting to write your own lower level control by examining how ROS works.